In [2]:
import pandas as pd

In [3]:
GDP = pd.read_csv("data/economic_indicators/GDP")
FEDFUNDS = pd.read_csv("data/economic_indicators/FEDFUNDS")
CPIAUCSL = pd.read_csv("data/economic_indicators/CPIAUCSL")
UNRATE = pd.read_csv("data/economic_indicators/UNRATE")
GS10 = pd.read_csv("data/economic_indicators/GS10")
INDPRO = pd.read_csv("data/economic_indicators/INDPRO")
PPIACO = pd.read_csv("data/economic_indicators/PPIACO")
RSXFS = pd.read_csv("data/economic_indicators/RSXFS")
HOUST = pd.read_csv("data/economic_indicators/HOUST")
PSAVERT = pd.read_csv("data/economic_indicators/PSAVERT")




In [4]:
indicators = [
            GDP, CPIAUCSL, UNRATE, FEDFUNDS, GS10,
        INDPRO, PPIACO, RSXFS, HOUST, PSAVERT
    ]
for ind in indicators:
    print(ind.shape)

(309, 2)
(928, 2)
(916, 2)
(838, 2)
(853, 2)
(1264, 2)
(1336, 2)
(388, 2)
(784, 2)
(783, 2)


In [5]:
for ind in indicators:
    ind.columns = ['Date', ind.columns[1]]

# Merging based on the 'Date' column
merged_df = GDP
for ind in indicators[1:]:
    merged_df = pd.merge(merged_df, ind, on='Date', how='outer')

# Check the shape of the merged DataFrame
print(merged_df.shape)

(1336, 11)


In [6]:
merged_df.dropna(inplace=True)


In [7]:
merged_df.shape

(129, 11)

In [8]:
merged_df.to_csv("./data/indicators.csv")

In [9]:
from data_collection.collect_num_data import get_economic_indicators, get_historical_prices, get_dividends_and_earnings


In [10]:
portfolio_csv = "portfolio_files/Apr-2025 portfolio.csv"
portfolio_df = pd.read_csv(portfolio_csv)
portfolio_symbols = portfolio_df["Symbol"]


In [11]:
import matplotlib.pyplot as plt


In [12]:
for symbol in portfolio_symbols:
    hist_data = get_historical_prices(symbol)
    try:
        hist_data.index = pd.to_datetime(hist_data.index)
        hist_data.index = hist_data.index.strftime('%Y-%m-%d')
    except Exception as e:
        print(e)
    hist_data.to_csv(f"./data/historical/{symbol}.csv")


In [13]:
tmp = get_historical_prices("GOOG")
tmp.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [14]:
indicators_df = pd.read_csv("data/indicators.csv")

In [15]:
indicators_df

,Unnamed: 0,Date,GDP,CPIAUCSL,UNRATE,FEDFUNDS,GS10,INDPRO,PPIACO,RSXFS,HOUST,PSAVERT
0,948,1992-01-01,6363.102,138.300,7.3,4.03,7.03,61.4898,115.600,146925.0,1176.0,9.5
1,951,1992-04-01,6470.763,139.400,7.4,3.73,7.48,62.9312,116.300,148032.0,1099.0,9.8
2,954,1992-07-01,6566.641,140.500,7.7,3.25,6.84,63.7408,117.900,150761.0,1139.0,9.5
3,957,1992-10-01,6680.803,141.700,7.3,3.10,6.59,64.0282,118.100,153521.0,1244.0,7.9
4,960,1993-01-01,6729.459,142.800,7.3,3.02,6.60,64.6256,118.000,157555.0,1210.0,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...
124,1320,2023-01-01,26813.601,300.356,3.4,4.33,3.53,102.5478,260.227,603496.0,1361.0,4.4
125,1323,2023-04-01,27063.012,303.032,3.4,4.83,3.46,103.1512,256.908,595397.0,1368.0,5.2
126,1326,2023-07-01,27610.128,304.628,3.5,5.12,3.90,103.1900,253.835,599037.0,1473.0,4.4
127,1329,2023-10-01,27956.998,307.531,3.8,5.33,4.80,102.5683,255.192,606596.0,1365.0,4.0


In [16]:
goog_df =  pd.read_csv("data/historical/GOOG.csv")
jpm_df =  pd.read_csv("data/historical/JPM.csv")
msft_df =  pd.read_csv("data/historical/MSFT.csv")
nvda_df =  pd.read_csv("data/historical/NVDA.csv")
voo_df =  pd.read_csv("data/historical/VOO.csv")

In [17]:
dataframes = {
    'GOOG': goog_df,
    'JPM': jpm_df,
    'MSFT':msft_df,
    'NVDA': nvda_df,
    'VOO':voo_df
}

In [18]:
combined_df = pd.concat([df.set_index('Date') for df in dataframes.values()], axis=1, keys=dataframes.keys())
combined_df.reset_index(inplace=True)

In [19]:
combined_df

Date      GOOG                                             \
                       Open      High       Low     Close       Volume   
0      2004-08-19  2.487833  2.588839  2.387325  2.496292  897427216.0   
1      2004-08-20  2.512960  2.713729  2.500273  2.694573  458857488.0   
2      2004-08-23  2.755276  2.823193  2.712983  2.721690  366857939.0   
3      2004-08-24  2.767466  2.776422  2.576649  2.608991  306396159.0   
4      2004-08-25  2.611230  2.686860  2.584361  2.637103  184645512.0   
...           ...       ...       ...       ...       ...          ...   
11208  2004-08-13       NaN       NaN       NaN       NaN          NaN   
11209  2004-08-16       NaN       NaN       NaN       NaN          NaN   
11210  2004-08-17       NaN       NaN       NaN       NaN          NaN   
11211  2004-08-18       NaN       NaN       NaN       NaN          NaN   
11212  2024-09-06       NaN       NaN       NaN       NaN          NaN   

              JPM                                      ...        NVDA  \
             Open        High         Low       Close  ...        Open   
0       22.305613   22.305613   22.032031   22.119345  ...    0.088895   
1       22.101878   22.585010   22.072772   22.497698  ...    0.088666   
2       22.555913   22.631585   22.392929   22.439495  ...    0.095926   
3       22.555910   22.614118   22.317254   22.462776  ...    0.097838   
4       22.468600   23.068152   22.468600   22.957554  ...    0.094933   
...           ...         ...         ...         ...  ...         ...   
11208   21.432476   21.496506   21.246208   21.461580  ...    0.078882   
11209   21.508150   22.002924   21.496508   21.968000  ...    0.079340   
11210   22.090242   22.264868   22.037854   22.078600  ...    0.081251   
11211   22.061137   22.334717   22.020391   22.305613  ...    0.084232   
11212  217.600006  218.740005  211.089996  212.315308  ...  108.040001   

                                                                VOO  \
             High         Low       Close        Volume        Open   
0        0.090500    0.087748    0.089124  7.244040e+08         NaN   
1        0.095697    0.088359    0.094398  1.199040e+09         NaN   
2        0.097455    0.095392    0.096462  8.000280e+08         NaN   
3        0.098296    0.091035    0.092869  9.394200e+08         NaN   
4        0.097531    0.093175    0.096844  7.213080e+08         NaN   
...           ...         ...         ...           ...         ...   
11208    0.080792    0.078194    0.080257  8.157000e+08         NaN   
11209    0.081480    0.078882    0.079493  5.317320e+08         NaN   
11210    0.084309    0.080487    0.083850  7.449120e+08         NaN   
11211    0.089430    0.083544    0.089430  8.724240e+08         NaN   
11212  108.150002  100.949997  102.355003  3.135027e+08  505.339996   

                                                      
             High         Low       Close     Volume  
0             NaN         NaN         NaN        NaN  
1             NaN         NaN         NaN        NaN  
2             NaN         NaN         NaN        NaN  
3             NaN         NaN         NaN        NaN  
4             NaN         NaN         NaN        NaN  
...           ...         ...         ...        ...  
11208         NaN         NaN         NaN        NaN  
11209         NaN         NaN         NaN        NaN  
11210         NaN         NaN         NaN        NaN  
11211         NaN         NaN         NaN        NaN  
11212  506.940002  495.799988  496.109985  4896278.0  

[11213 rows x 26 columns]

In [20]:
indicators_df['Date'] = pd.to_datetime(indicators_df['Date'])
combined_df['Date'] = pd.to_datetime(combined_df['Date'])


In [21]:
# Flatten MultiIndex columns
combined_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in combined_df.columns]

# Ensure 'Date' column is correctly named
if 'Date_' in combined_df.columns:
    combined_df.rename(columns={'Date_': 'Date'}, inplace=True)


In [22]:
full_df = pd.merge(combined_df, indicators_df, on='Date', how='left')


In [23]:
full_df

,Date,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,...,GDP,CPIAUCSL,UNRATE,FEDFUNDS,GS10,INDPRO,PPIACO,RSXFS,HOUST,PSAVERT
0,2004-08-19,2.487833,2.588839,2.387325,2.496292,897427216.0,22.305613,22.305613,22.032031,22.119345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-08-20,2.512960,2.713729,2.500273,2.694573,458857488.0,22.101878,22.585010,22.072772,22.497698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-08-23,2.755276,2.823193,2.712983,2.721690,366857939.0,22.555913,22.631585,22.392929,22.439495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-08-24,2.767466,2.776422,2.576649,2.608991,306396159.0,22.555910,22.614118,22.317254,22.462776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-08-25,2.611230,2.686860,2.584361,2.637103,184645512.0,22.468600,23.068152,22.468600,22.957554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11208,2004-08-13,NaN,NaN,NaN,NaN,NaN,21.432476,21.496506,21.246208,21.461580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11209,2004-08-16,NaN,NaN,NaN,NaN,NaN,21.508150,22.002924,21.496508,21.968000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11210,2004-08-17,NaN,NaN,NaN,NaN,NaN,22.090242,22.264868,22.037854,22.078600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11211,2004-08-18,NaN,NaN,NaN,NaN,NaN,22.061137,22.334717,22.020391,22.305613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)


In [25]:
full_df_imputed = full_df.copy()
full_df_imputed.iloc[:, 1:] = imputer.fit_transform(full_df_imputed.iloc[:, 1:])


In [26]:
full_df_imputed

,Date,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,...,GDP,CPIAUCSL,UNRATE,FEDFUNDS,GS10,INDPRO,PPIACO,RSXFS,HOUST,PSAVERT
0,2004-08-19,2.487833,2.588839,2.387325,2.496292,897427216.0,22.305613,22.305613,22.032031,22.119345,...,7502.7374,149.6000,6.36,4.056,6.218,70.67188,121.60,175999.0,1389.0,7.34
1,2004-08-20,2.512960,2.713729,2.500273,2.694573,458857488.0,22.101878,22.585010,22.072772,22.497698,...,7502.7374,149.6000,6.36,4.056,6.218,70.67188,121.60,175999.0,1389.0,7.34
2,2004-08-23,2.755276,2.823193,2.712983,2.721690,366857939.0,22.555913,22.631585,22.392929,22.439495,...,7742.8652,152.2800,5.98,4.462,6.238,72.77582,123.46,184002.0,1411.8,6.92
3,2004-08-24,2.767466,2.776422,2.576649,2.608991,306396159.0,22.555910,22.614118,22.317254,22.462776,...,7472.4578,149.4000,6.32,4.038,6.254,70.72818,121.60,175757.4,1357.4,7.40
4,2004-08-25,2.611230,2.686860,2.584361,2.637103,184645512.0,22.468600,23.068152,22.468600,22.957554,...,8312.6666,157.3200,5.30,5.004,6.078,77.61714,125.36,197787.2,1476.2,6.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11208,2004-08-13,9.351367,9.432421,9.264442,9.322409,247552422.0,21.432476,21.496506,21.246208,21.461580,...,7472.4578,149.4000,6.32,4.038,6.254,70.72818,121.60,175757.4,1357.4,7.40
11209,2004-08-16,10.168621,10.312218,9.998105,10.138369,470500164.6,21.508150,22.002924,21.496508,21.968000,...,8948.9382,162.5600,5.40,4.176,5.496,80.62768,127.12,211989.2,1510.4,6.16
11210,2004-08-17,8.640643,8.696619,8.567551,8.629448,213490024.8,22.090242,22.264868,22.037854,22.078600,...,8545.1832,158.7000,5.10,4.938,5.610,79.86140,124.56,202446.4,1524.8,6.34
11211,2004-08-18,8.855044,8.939680,8.767921,8.838277,286163806.6,22.061137,22.334717,22.020391,22.305613,...,9075.4106,163.0400,5.40,4.032,5.376,81.29254,127.44,214431.0,1578.6,6.02


In [27]:
full_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11213 entries, 0 to 11212
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         11213 non-null  datetime64[ns]
 1   GOOG_Open    11213 non-null  float64       
 2   GOOG_High    11213 non-null  float64       
 3   GOOG_Low     11213 non-null  float64       
 4   GOOG_Close   11213 non-null  float64       
 5   GOOG_Volume  11213 non-null  float64       
 6   JPM_Open     11213 non-null  float64       
 7   JPM_High     11213 non-null  float64       
 8   JPM_Low      11213 non-null  float64       
 9   JPM_Close    11213 non-null  float64       
 10  JPM_Volume   11213 non-null  int64         
 11  MSFT_Open    11213 non-null  float64       
 12  MSFT_High    11213 non-null  float64       
 13  MSFT_Low     11213 non-null  float64       
 14  MSFT_Close   11213 non-null  float64       
 15  MSFT_Volume  11213 non-null  float64       
 16  NVDA

In [30]:
import numpy as np

# Feature Engineering


In [31]:
stocks = ['GOOG', 'JPM', 'MSFT', 'NVDA', 'VOO']
economic_indicators = ['GDP', 'CPIAUCSL', 'UNRATE', 'FEDFUNDS', 'GS10', 'INDPRO', 'PPIACO', 'RSXFS', 'HOUST', 'PSAVERT']


In [34]:
# Log Returns for each stock (Relevant for modeling risk and capturing the stock's behavior over time)
for stock in stocks:
    full_df_imputed[f'{stock}_LogReturn'] = np.log(full_df_imputed[f'{stock}_Close'] / full_df_imputed[f'{stock}_Close'].shift(1))


In [35]:
full_df_imputed.tail()

,Date,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,...,INDPRO,PPIACO,RSXFS,HOUST,PSAVERT,GOOG_LogReturn,JPM_LogReturn,MSFT_LogReturn,NVDA_LogReturn,VOO_LogReturn
11208,2004-08-13,9.351367,9.432421,9.264442,9.322409,247552422.0,21.432476,21.496506,21.246208,21.461580,...,70.72818,121.60,175757.4,1357.4,7.40,-1.281036,0.000271,0.005195,0.040822,-0.319949
11209,2004-08-16,10.168621,10.312218,9.998105,10.138369,470500164.6,21.508150,22.002924,21.496508,21.968000,...,80.62768,127.12,211989.2,1510.4,6.16,0.083906,0.023322,0.002587,-0.009566,0.190687
11210,2004-08-17,8.640643,8.696619,8.567551,8.629448,213490024.8,22.090242,22.264868,22.037854,22.078600,...,79.86140,124.56,202446.4,1524.8,6.34,-0.161147,0.005022,-0.001478,0.053358,-0.308864
11211,2004-08-18,8.855044,8.939680,8.767921,8.838277,286163806.6,22.061137,22.334717,22.020391,22.305613,...,81.29254,127.44,214431.0,1578.6,6.02,0.023911,0.010230,0.015043,0.064421,-0.052083
11212,2024-09-06,142.773119,144.296534,142.005615,143.625082,15808120.0,217.600006,218.740005,211.089996,212.315308,...,94.29818,188.04,403063.4,1352.4,9.36,2.788114,2.253234,3.155913,7.042751,0.867377


In [36]:
# Rolling Volatility for each stock (Relevant for identifying periods of high/low risk)
for stock in stocks:
    full_df_imputed[f'{stock}_RollingVolatility_30d'] = full_df_imputed[f'{stock}_LogReturn'].rolling(window=30).std()


In [37]:
# Percentage changes in economic indicators (Capture macroeconomic shifts and trends, important for risk modeling)
for indicator in economic_indicators:
    full_df_imputed[f'{indicator}_PctChange'] = full_df_imputed[indicator].pct_change()


In [38]:
# Moving averages of stock closing prices (Relevant for identifying trends and stock behavior over time)
for stock in stocks:
    full_df_imputed[f'{stock}_MA10'] = full_df_imputed[f'{stock}_Close'].rolling(window=10).mean()  # Short-term trend
    full_df_imputed[f'{stock}_MA200'] = full_df_imputed[f'{stock}_Close'].rolling(window=200).mean()  # Long-term trend


In [39]:
# Lag features for log returns and economic indicators (Relevant for capturing past behavior and predicting future movements)
lags = [1, 5, 10]
for stock in stocks:
    for lag in lags:
        full_df_imputed[f'{stock}_LogReturn_Lag{lag}'] = full_df_imputed[f'{stock}_LogReturn'].shift(lag)

for indicator in economic_indicators:
    for lag in lags:
        full_df_imputed[f'{indicator}_PctChange_Lag{lag}'] = full_df_imputed[f'{indicator}_PctChange'].shift(lag)

In [40]:
# Maximum drawdown for each stock (Capture the largest loss from peak to trough, relevant for modeling worst-case scenarios)
for stock in stocks:
    full_df_imputed[f'{stock}_MaxDrawdown'] = full_df_imputed[f'{stock}_Close'].rolling(window=252, min_periods=1).apply(lambda x: (x.max() - x.min()) / x.max(), raw=True)

In [41]:
# Rolling correlation between stock log returns (Relevant for understanding stock relationships, which can impact portfolio risk)
for i, stock1 in enumerate(stocks):
    for stock2 in stocks[i+1:]:
        full_df_imputed[f'{stock1}_{stock2}_RollingCorr_30d'] = full_df_imputed[f'{stock1}_LogReturn'].rolling(window=30).corr(full_df_imputed[f'{stock2}_LogReturn'])

In [42]:
# Rolling correlation between stock returns and economic indicators (Capture how economic factors affect stock movements)
for stock in stocks:
    for indicator in economic_indicators:
        full_df_imputed[f'{stock}_{indicator}_RollingCorr_30d'] = full_df_imputed[f'{stock}_LogReturn'].rolling(window=30).corr(full_df_imputed[f'{indicator}_PctChange'])


C:\Users\Rj\AppData\Local\Temp\ipykernel_25360\1290757663.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df_imputed[f'{stock}_{indicator}_RollingCorr_30d'] = full_df_imputed[f'{stock}_LogReturn'].rolling(window=30).corr(full_df_imputed[f'{indicator}_PctChange'])
C:\Users\Rj\AppData\Local\Temp\ipykernel_25360\1290757663.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df_imputed[f'{stock}_{indicator}_RollingCorr_30d'] = full_df_imputed[f'{stock}_LogReturn'].rolling(window=30).corr(full_df_imputed[f'{indicator}_P

In [43]:
# Outlier detection (Flagging large returns for risk identification, which may indicate unusual events or risks)
for stock in stocks:
    full_df_imputed[f'{stock}_Outliers'] = np.abs(full_df_imputed[f'{stock}_LogReturn']) > (full_df_imputed[f'{stock}_LogReturn'].mean() + 3 * full_df_imputed[f'{stock}_LogReturn'].std())

C:\Users\Rj\AppData\Local\Temp\ipykernel_25360\839149101.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df_imputed[f'{stock}_Outliers'] = np.abs(full_df_imputed[f'{stock}_LogReturn']) > (full_df_imputed[f'{stock}_LogReturn'].mean() + 3 * full_df_imputed[f'{stock}_LogReturn'].std())
C:\Users\Rj\AppData\Local\Temp\ipykernel_25360\839149101.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df_imputed[f'{stock}_Outliers'] = np.abs(full_df_imputed[f'{stock}_LogReturn']) > (full_df_imputed[f'{stock}_LogReturn'].mean()

In [47]:
full_df_imputed.isnull().sum()

In [52]:
full_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11213 entries, 0 to 11212
Columns: 182 entries, Date to VOO_Outliers
dtypes: bool(5), datetime64[ns](1), float64(175), int64(1)
memory usage: 15.2 MB


In [53]:
# Drop NaN values caused by the shift and rolling operations
full_df_imputed.dropna(inplace=True)

In [57]:
full_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9603 entries, 199 to 11212
Columns: 182 entries, Date to VOO_Outliers
dtypes: bool(5), datetime64[ns](1), float64(175), int64(1)
memory usage: 13.1 MB


In [59]:
full_df_imputed.columns

Index(['Date', 'GOOG_Open', 'GOOG_High', 'GOOG_Low', 'GOOG_Close',
       'GOOG_Volume', 'JPM_Open', 'JPM_High', 'JPM_Low', 'JPM_Close',
       ...
       'VOO_INDPRO_RollingCorr_30d', 'VOO_PPIACO_RollingCorr_30d',
       'VOO_RSXFS_RollingCorr_30d', 'VOO_HOUST_RollingCorr_30d',
       'VOO_PSAVERT_RollingCorr_30d', 'GOOG_Outliers', 'JPM_Outliers',
       'MSFT_Outliers', 'NVDA_Outliers', 'VOO_Outliers'],
      dtype='object', length=182)

In [27]:
from sklearn.preprocessing import StandardScaler

# Select relevant features (excluding 'Date')
features = full_df_imputed.columns.difference(['Date'])
X = full_df_imputed[features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create a DataFrame with scaled features
X_scaled_df = pd.DataFrame(X_scaled, columns=features)


In [41]:

def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps].values)
    return np.array(Xs), np.array(ys)

time_steps = 60
closing_prices = full_df_imputed[[col for col in full_df_imputed.columns if 'Close' in col]]
X_sequences, y_sequences = create_sequences(X_scaled_df, closing_prices, time_steps)

# Split into training and testing sets
train_size = int(len(X_sequences) * 0.8)
X_train, X_test = X_sequences[:train_size], X_sequences[train_size:]
y_train, y_test = y_sequences[:train_size], y_sequences[train_size:]


In [42]:
X_train.shape

(8868, 60, 36)

In [43]:
y_train.shape

(8868, 5)

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(y_train.shape[1]))  # Output layer for multiple features (closing prices of all tickers)




E:\Personal\Risk_management_model\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [46]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 15428.0635 - val_loss: 5356.6655
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 6457.2900 - val_loss: 3824.7622
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 4373.1182 - val_loss: 3077.4495
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 2895.7319 - val_loss: 1871.8192
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 2260.6367 - val_loss: 1414.3041
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 1757.3263 - val_loss: 1352.1914
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 1377.3611 - val_loss: 1295.5582
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 1212.2306 - val_loss: 1256.3359
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 1169.0806 - val_loss: 1272.6152
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 1064.1920 - val_loss: 1319.8784


In [47]:
model.evaluate(X_test, y_test)


70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1607.1604


2190.14453125

In [48]:
predictions = model.predict(X_test)
returns = pd.DataFrame(predictions - y_test, columns=closing_prices.columns)  # Calculate returns from predictions

def calculate_var(returns, confidence_level=0.95):
    sorted_returns = np.sort(returns, axis=0)
    index = int((1 - confidence_level) * sorted_returns.shape[0])
    return sorted_returns[index]

def calculate_cvar(returns, var):
    return np.mean(returns[returns <= var], axis=0)

var_95 = calculate_var(returns, 0.95)
cvar_95 = calculate_cvar(returns, var_95)

print(f"Value at Risk (95%): {var_95}")
print(f"Conditional Value at Risk (95%): {cvar_95}")


70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Value at Risk (95%): [ -31.19716563  -32.22780418  -49.56879044  -47.06499362 -197.95496368]
Conditional Value at Risk (95%): GOOG_Close    -55.416149
JPM_Close     -37.271633
MSFT_Close    -60.019888
NVDA_Close    -83.835048
VOO_Close    -265.614849
dtype: float64
